# TTC Implementation

## Modifications required

* adapt to expect xx input band feature stack. currently expects 17 channels
* change the final output layer to support multi-class segmentation with a softmax activation function. currently is sigmoid
* adjust input patch size. currently expects 512x512
* adjust loss function to be categorical cross-entropy (or Dice loss for imbalanced classes). currently is binary cross-entropy. confirm whether to consider weighted loss function.
* model deployment - how large of an area?
* model validation - necessary?

### Updates reflected in `modified_unet.py`
* Removed ConvGRU: classification will be done on a single composite images (not monthly time series). No need for RNN/time layers.
* Removed SSE and DropBlock: Starting with a simple model. Attention and regularization can be added later if needed.
* Switched Binary to Multiclass Output: Instead of "tree vs no-tree", your model now predicts 4 classes: Background, Monoculture, Agroforestry, Natural.
* Simplified U-Net Encoder-Decoder: Clean, readable U-Net blocks: Conv → ReLU → BatchNorm with skip connections.
* Modified Input Channels: Original model expected 17 bands (Sentinel only); your model expects 94 bands (Sentinel + Texture + Tree Features).

In [ ]:
# how to modify the training data